# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [3]:
# as dict
dict(vocs)

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [4]:
#  re-parse dict
vocs2 = VOCS.from_dict(dict(vocs))

In [5]:
# Check that these are the same
vocs2 == vocs

True

In [6]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [7]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [8]:
# json
vocs.to_json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [9]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,456.216127,-0.066455,1234,456.149672,456.282583,912.299344,912.565166
6,44.722740,0.695131,1234,45.417872,44.027609,90.835743,88.055218
7,806.198239,0.466200,1234,806.664440,805.732039,1613.328880,1611.464078
8,883.772499,-0.259895,1234,883.512603,884.032394,1767.025206,1768.064788
9,441.842996,0.128248,1234,441.971244,441.714747,883.942488,883.429495
10,98.827816,0.910703,1234,99.738519,97.917113,199.477039,195.834227
11,556.535400,-0.235057,1234,556.300343,556.770457,1112.600685,1113.540915
12,43.541003,-0.269458,1234,43.271545,43.810461,86.543090,87.620922
13,668.657791,0.678677,1234,669.336467,667.979114,1338.672935,1335.958228
14,113.047039,-0.016677,1234,113.030362,113.063716,226.060724,226.127431


In [10]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [11]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-456.149672,456.282583
6,-45.417872,44.027609
7,-806.664440,805.732039
8,-883.512603,884.032394
9,-441.971244,441.714747
10,-99.738519,97.917113
11,-556.300343,556.770457
12,-43.271545,43.810461
13,-669.336467,667.979114
14,-113.030362,113.063716


In [12]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-456.149672,456.282583
6,-45.417872,44.027609
7,-806.664440,805.732039
8,-883.512603,884.032394
9,-441.971244,441.714747
10,-99.738519,97.917113
11,-556.300343,556.770457
12,-43.271545,43.810461
13,-669.336467,667.979114
14,-113.030362,113.063716


In [13]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-456.14967195,  456.28258283],
       [ -45.41787159,   44.02760877],
       [-806.66443978,  805.73203892],
       [-883.5126031 ,  884.03239408],
       [-441.97124401,  441.7147473 ],
       [ -99.73851944,   97.91711345],
       [-556.30034275,  556.7704574 ],
       [ -43.27154505,   43.81046097],
       [-669.33646733,  667.97911407],
       [-113.03036191,  113.06371556]])

In [14]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,910.299344,-912.565166
6,88.835743,-88.055218
7,1611.328880,-1611.464078
8,1765.025206,-1768.064788
9,881.942488,-883.429495
10,197.477039,-195.834227
11,1110.600685,-1113.540915
12,84.543090,-87.620922
13,1336.672935,-1335.958228
14,224.060724,-226.127431


In [15]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


In [16]:
# normalize inputs to unit domain [0,1]
normed_data = vocs.normalize_inputs(data)
normed_data

,a,b
5,0.456216,0.466772
6,0.044723,0.847566
7,0.806198,0.733100
8,0.883772,0.370052
9,0.441843,0.564124
10,0.098828,0.955351
11,0.556535,0.382471
12,0.043541,0.365271
13,0.668658,0.839338
14,0.113047,0.491662


In [17]:
# and denormalize
vocs.denormalize_inputs(normed_data)

,a,b
5,456.216127,-0.066455
6,44.722740,0.695131
7,806.198239,0.466200
8,883.772499,-0.259895
9,441.842996,0.128248
10,98.827816,0.910703
11,556.535400,-0.235057
12,43.541003,-0.269458
13,668.657791,0.678677
14,113.047039,-0.016677


# Error handling

In [18]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [19]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [20]:
data['a']  = np.nan

In [21]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [22]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [23]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
